# Semantic-Kernel Bot

- A bot keeps a history of the conversation

## Load Required .NET Packages

In [17]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: dotenv.net"

using System.Net.Http;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ImageGeneration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

Installed Packages dotenv.net, 3.1.2 Microsoft.SemanticKernel, 0.17.230626.1-preview

## Load Environment Variables

In [29]:
DotEnv.Load();
var depoymentName = Environment.GetEnvironmentVariable("DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("API_KEY");
Console.WriteLine($"Using deployment: {depoymentName} at {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at https://alemorfr.openai.azure.com/ with key 32c87...


## Get an instance for a HttpClient

**> Note:** This is not necessary, but it may help prevent port exhaustion issues.

In [ ]:
// Note: This may be good practice depending on your use case. Otherwise, an HttpClient will be created for you.
var httpClient = new HttpClient();

## Build a Kernel

- Registration: `WithAzureChatCompletionServices` registers an ICompletion service with the kernel
- Retries: Kernel API calls come with default retry logic for timeouts and throttling.

In [38]:
var kernel = new KernelBuilder()
    .WithAzureChatCompletionService(depoymentName,endpoint,apiKey, httpClient: httpClient)
    .Build();

var systemMessage = "You are a general assistant. Say Hello, I'm your friendly bot. How can I assist you today?";

## Get the registered service from the kernel and keep a conversation (a history)

In [36]:
// Get an IChatCompletion service from the kernel
var chatGPT = kernel.GetService<IChatCompletion>();

// Keep a history
var conversation = (OpenAIChatHistory)chatGPT.CreateNewChat(systemMessage);

## Call the bot for the first time

In [37]:
string assistantReply = await chatGPT.GenerateMessageAsync(conversation, new ChatRequestSettings());
Console.WriteLine($"\nBot: {assistantReply}");
conversation.AddAssistantMessage(assistantReply);


Bot: Hello, I'm your friendly bot. How can I assist you today?


## Enter the while loop

In [32]:
while (true) {
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {userMessage}");
    conversation.AddUserMessage(userMessage);
    if (userMessage == "history") {
        foreach (var message in chat.Messages) {
            Console.WriteLine(message.Role+ ": " + message.Content);
        }
        continue;
    }
    string assistantReply = await chatGPT.GenerateMessageAsync(conversation, new ChatRequestSettings());
    // or
    //IReadOnlyList<ITextResult> completionResults = await chatGPT.GetCompletionsAsync(conversation, new CompleteRequestSettings() { MaxTokens = 100, Temperature = 0.4, TopP = 1 });
    //Console.WriteLine(await completionResults[0].GetCompletionAsync());
    //Console.WriteLine(completionResults[0].ModelResult.GetOpenAIChatResult().Usage.AsJson());
    //Console.WriteLine();

    Console.WriteLine($"\nBot: {assistantReply}");
    // Add the message to the conversation history
    conversation.AddAssistantMessage(assistantReply);
}

User: What is the speed of light?

Bot: The speed of light is approximately 299,792,458 meters per second (or about 670,616,629 miles per hour). It is the fastest known speed in the universe and is used as a fundamental constant in many areas of physics and astronomy.
User: What is the distance to the sun?

Bot: The average distance from the Earth to the Sun is about 93 million miles (149.6 million kilometers). This distance is also known as an astronomical unit (AU) and is used as a standard unit of measurement in astronomy.
User: List some top restaurants in Miami.

Bot: Sure, here are some top restaurants in Miami:

1. Joe's Stone Crab
2. Yardbird Southern Table & Bar
3. Versailles Restaurant
4. La Mar by Gastón Acurio
5. KYU
6. Zuma
7. Michael's Genuine Food & Drink
8. Mandolin Aegean Bistro
9. Alter
10. The Bazaar by José Andrés

Please note that this is not an exhaustive list and there are many other great restaurants in Miami as well.
User: List some more.

Bot: Sure, here are s

Error: Input request cancelled

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 79
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 47
   at Submission#33.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)